In [ ]:
# Dependencies
import csv
import pandas as pd
import numpy as np


# You must include a written description of three observable trends based on the data.
# Few observations can be made out of the dataset given:

# 1/ Fist and foremost, this game is address for a young public given the bin breakdown by age. Indeed most of the purchases are targeting a teen public (15/19 years old)
# Within the teen large bracket (10/19 years old), it is worth noting that the (15/19 years old) generates almost 3 times more revenue than the (10/14 years old) given the fact that the former bracket is likely to have more money.

# 2/ The frequency distribution analysis shows that less and less players are likely to buy items as they grow older.

# 3/ Males make more than 80% of the players and as well 80% of the revenue.

# 4/ The most profitable items are for the given set roughly twice more expensive than the average ones.

In [571]:
# The path to the json file
file = "generated_data/purchase_data.json"
file_df = pd.read_json(file)
file_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [572]:
file_df.dtypes

Age            int64
Gender        object
Item ID        int64
Item Name     object
Price        float64
SN            object
dtype: object

In [573]:
# Check if missing data
file_df.count()

Age          780
Gender       780
Item ID      780
Item Name    780
Price        780
SN           780
dtype: int64

In [574]:
# Display an overview of the database
file_df.describe()

,Age,Item ID,Price
count,780.000000,780.000000,780.000000
mean,22.729487,91.293590,2.931192
std,6.930604,52.707537,1.115780
min,7.000000,0.000000,1.030000
25%,19.000000,44.000000,1.960000
50%,22.000000,91.000000,2.880000
75%,25.000000,135.000000,3.910000
max,45.000000,183.000000,4.950000


In [594]:
# Player Count
players_count = file_df.loc[:,["Gender","SN","Age"]]
players_count = players_count.drop_duplicates()
players_count_df = players_count.count()[0]

pd.DataFrame({"Total Players":[players_count_df]})


,Total Players
0,573


In [595]:
# Display the unique items from the list of items
items_count=len(file_df["Item ID"].unique())
items_count

183

In [596]:
# Display an the average purchase price from the list of items
avg_purchase_price = file_df["Price"].mean()
avg_purchase_price

2.931192307692303

In [597]:
# Display the total number of purchase 
total_purchase_count = file_df["Price"].count()
total_purchase_count

780

In [598]:
# Display the total revenue
total_purchase_value = file_df["Price"].sum()
total_purchase_value

2286.33

In [599]:
# Purchasing Analysis (total)
purchases_summary_table = pd.DataFrame({"Number of Unique Items":items_count,
                                 "Average Price": [avg_purchase_price],
                                 "Number of Purchases":[total_purchase_count],
                                 "Total revenue":[total_purchase_value]}, 
                                       columns = ["Number of Unique Items",
                                                  "Average Price",
                                                  "Number of Purchases",
                                                 "Total revenue"])

purchases_summary_table.style.format({"Average Price":"${:.2f}","Total revenue": "${:.2f}"})

,Number of Unique Items,Average Price,Number of Purchases,Total revenue
0,183,$2.93,780,$2286.33


In [600]:
# Gender demographics
gender_breakdown = players_count["Gender"].value_counts()
gender_breakdown_perc = (gender_breakdown/players_count_df)*100


gender_summary_table = pd.DataFrame({"Total Count": gender_breakdown,"Percentage of Players":gender_breakdown_perc}, columns = ["Percentage of Players","Total Count"])
gender_summary_table


gender_summary_table.style.format({"Percentage of Players":"{:.2f}%","Total Count":"{:.0f}"})


,Percentage of Players,Total Count
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclosed,1.40%,8


In [601]:
# Purchasing Analysis (gender)
gender_purchase_count = file_df.groupby(["Gender"]).count()["Price"].rename("Purchase Count")
gender_purchase_count 
gender_purchase_value = file_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_purchase_value 
gender_avg_purchase = file_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_avg_purchase
normalized_total = gender_purchase_value / gender_breakdown
normalized_total


gender_breakdown_summary_table = pd.DataFrame({"Purchase Count": gender_purchase_count,
                                               "Total Purchase Value":gender_purchase_value,
                                               "Average Purchase Price":gender_avg_purchase,
                                               "Normalized Totals":normalized_total
                                              }, columns = ["Purchase Count",
                                                            "Average Purchase Price",
                                                            "Total Purchase Value",
                                                           "Normalized Totals"])

gender_breakdown_summary_table


gender_breakdown_summary_table.style.format({"Total Purchase Value":"${:.2f}","Average Purchase Price":"${:.2f}", "Normalized Totals":"${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,$1867.68,$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [602]:
# Create the bins in which Data will be held
age_bins_values = [0,9,14,19,24,29,34,39,44,100]
# Create the names for the different bins
age_bins_names = ["<10","10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45+"]
# Cut age and place the scores into bins
file_df["Age Bins"] = pd.cut(file_df["Age"],age_bins_values, labels=age_bins_names)

file_df.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Bins
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [603]:
# Create a GroupBy object based upon "Age Bins"
player_bin_group = file_df.groupby("Age Bins")

# # Find how many rows fall into each bin
# print(player_bin_group["Age Bins"].count())

# Calculate the number and % by age bins
# player_count_bin = players_count["Age Bins"].value_counts()
# player_count_bin

player_bin_group = file_df.groupby("Age Bins")

# player_perc_bin
player_perc_bin = ((player_count_bin/players_count_df)*100).round(2)

# Format the dataframe
player_bin_summary_table = pd.DataFrame({"Age":["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49"],
                                         "Total Count": player_count_bin,
                                        "Percentage of Players":player_perc_bin}, 
                                        columns = ["Age", "Percentage of Players","Total Count"])

# Order the results by Age bin
player_bin_summary_table

# Print table
player_bin_summary_table.set_index("Age")



,Percentage of Players,Total Count
Age,,
<10,45.20,259
10-14,17.45,100
15-19,15.18,87
20-24,8.20,47
25-29,4.71,27
30-34,4.01,23
35-39,3.32,19
40-44,1.75,10
45-49,0.17,1


In [604]:
# Bin Purchase by Age
file_df["Age Bins"] = pd.cut(file_df["Age"],age_bins_values, labels=age_bins_names)

# Calculate Purchase Count, Total Sum and Avg Price
age_total_purchase = file_df.groupby(["Age Bins"]).sum()["Price"].rename("Total Purchase Value")
age_avg_purchase = file_df.groupby(["Age Bins"]).mean()["Price"].rename("Average Purchase Price")
age_counts_purchase = file_df.groupby(["Age Bins"]).count()["Price"].rename("Purchase Count")

# Calculate Normalized totals
normalized_total = age_total_purchase / player_bin_summary_table["Total Count"]

# Format the dataframe
player_data_summary_table = pd.DataFrame({"Age": ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49"],
                                          "Purchase Count": age_counts_purchase,
                                          "Average Purchase Price":age_avg_purchase,
                                          "Total Purchase Value":age_total_purchase,
                                          "Normalized Totals":normalized_total
                                         }, columns = ["Age","Purchase Count",
                                                       "Average Purchase Price",
                                                       "Total Purchase Value",
                                                      "Normalized Totals"])
# Index Age
player_age_summary_final = player_data_summary_table.set_index("Age")

# Format the results
player_age_summary_final.style.format({"Average Purchase Price":"${:.2f}","Total Purchase Value":"${:.2f}", "Normalized Totals":"${:.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age,,,,
<10,35,$2.77,$96.95,$4.22
10-14,133,$2.91,$386.42,$3.86
15-19,336,$2.91,$978.77,$3.78
20-24,125,$2.96,$370.33,$4.26
25-29,64,$3.08,$197.25,$4.20
30-34,42,$2.84,$119.40,$4.42
35-39,16,$3.19,$51.03,$5.10
40-44,1,$2.72,$2.72,$2.72
45-49,28,$2.98,$83.46,$4.39


In [605]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

sn_purchase_value=file_df.groupby(["SN"])['Price'].sum().rename("Total Purchase Value")
sn_purchase_count=file_df.groupby(["SN"])['Price'].count().rename("Purchase Count")
sn_purchase_avg=file_df.groupby(["SN"])['Price'].mean().rename("Average Purchase Price")

# Convert to DataFrame
spenders_data = pd.DataFrame({"Total Purchase Value":sn_purchase_value,"Purchase Count":sn_purchase_count,"Average Purchase Price":sn_purchase_avg},columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"])

spenders_data
                             
spenders_data_summary = spenders_data.sort_values("Total Purchase Value", ascending=False).head()
                             
# Format the results
spenders_data_summary.style.format({"Total Purchase Value":"${:.2f}","Average Purchase Price":"${:.2f}"})


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


In [606]:
#Identify the 5 most popular items by purchase count, then list (in a table):
item_data = file_df.loc[:,["Item ID", "Item Name", "Price"]]

pop_item_purchase = item_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
pop_item_count = item_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")
pop_item_avg = item_data.groupby(["Item ID", "Item Name"]).mean()["Price"].rename("Item Price")

# Convert to DataFrame
pop_item_data = pd.DataFrame({"Total Purchase Value":pop_item_purchase,
                              "Purchase Count":pop_item_count,
                              "Item Price":pop_item_avg},
                             columns = ["Purchase Count","Item Price","Total Purchase Value"])

pop_item_data.head()

pop_item_data_summary = pop_item_data.sort_values("Purchase Count", ascending=False).head()

# Format the results
pop_item_data_summary.style.format({"Total Purchase Value":"${:.2f}","Item Price":"${:.2f}"})


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [607]:
#Identify the 5 most profitable items by total purchase value, then list (in a table):
pop_item_data_pv_summary = pop_item_data.sort_values("Total Purchase Value", ascending=False).head()

# Format the results
pop_item_data_pv_summary.style.format({"Total Purchase Value":"${:.2f}","Item Price":"${:.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
